## Import Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Scraping Dataset

In [ ]:
!pip install google-play-scraper

Beberapa fungsi `google_play_scraper`:
* `app`: Menampilkan detail aplikasi
* `reviews`: Menampilkan komentar dari pengguna aplikasi
* `permission`: Menampilkan Izin yang diperlukan untuk menjalankan aplikasi
* `search`: Mencari aplikasi yang ada pada goole play dengan kata kunci yang kita inginkan

In [ ]:
from google_play_scraper import reviews, Sort

Opsi yang terdapat pada fungsi `reviews`:
1. ID aplikasi yang ada pada Google Play, didapat dari link aplikasi pada Google Play setelah sama dengan(=).
Contoh https://play.google.com/store/apps/details?id=com.ea.game.pvzfree_row maka ID aplikasinya adalah com.ea.game.pvzfree_row
2. `lang`: Bahasa yang digunakan dalam ulasan.
[Reverensi singkatan nama negara]('https://www.iban.com/country-codes')
3. `country`: Asal negara ulasan.
[Reverensi singkatan nama negara]('https://www.iban.com/country-codes')
4. `sort`: Mode pengurutan ulasan. Bisa menggunakan `Sort.MOST_RELEVANT` (Paling relevan) atau `Sort.NEWEST` (Paling baru).
5. `count`: Jumlah ulasan yang ingin diambil. Jika menggunakan script `reviews_all`, maka akan mengambil semua ulasan yang ada dan mengabaikan jumlah `count`.
6. `filter_score_with`: Rating ulasan (1 sd 5), artinya apabila diisi 1 maka ulasan yang diambil hanya ulasan pada rating 1, atau bisa diisi None untuk mengambil ulasan pada semua rating.

source : https://pypi.org/project/google-play-scraper/




In [ ]:
scrapdata, continuation_token = reviews(
    'com.ea.game.pvzfree_row',
    lang='id',
    country='id',
    sort=Sort.MOST_RELEVANT,
    count = 10000,
    filter_score_with = 1
)

**Menyimpan Ulasan dalam file CSV**

Sebagaimana diketahui Pandas merupakan paket python yang memberikan kemudahan dalam memproses dan menganalisa data. Biasanya data-data yang telah diolah disimpan dalam file, dan yang paling umum adalah file dengan format csv atau (comma separated value).

Untuk melakukan export Pandas DataFrame ke file csv, Pandas memiliki berbagai cara, diantaranya dengan memanfaatkan library `csv` atau menggunakan fungsi `.to_csv`.

source:

https://hub.idbigdata.com/sigit-prasetyo/export-pandas-dataframe-sebagai-file-csv--47

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html

In [ ]:
# Menyimpan data dengan library csv

import csv
with open('ulasan_aplikasi.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Review'])  # Menulis header kolom
    for review in scrapdata:
        writer.writerow([review['content']])  # Menulis konten ulasan ke dalam file CSV

In [ ]:
#Menyimpan data dengan fungsi .to_csv

scrap_df = pd.DataFrame(scrapdata)
scrap_df.to_csv('ulasan_apk.csv', # Nama file yang akan disimpan (diakhiri dengan .csv)
                index = False,   # Apakah index data akan ikut dimasukan kedalam file csv?
                sep = ',',       # Tanda yang memisahkan antar kolom (default: ,)
                header = True,   # Apakah header atau nama kolom disertakan pada file csv?
                mode = 'w'       # Mode pembuatan file memotong jika sudah ada
                )

## Loading Dataset

In [ ]:
scrap_df.info()
print('Jumlah nilai terdupliasi: ', scrap_df.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              10000 non-null  object        
 1   userName              10000 non-null  object        
 2   userImage             10000 non-null  object        
 3   content               10000 non-null  object        
 4   score                 10000 non-null  int64         
 5   thumbsUpCount         10000 non-null  int64         
 6   reviewCreatedVersion  3364 non-null   object        
 7   at                    10000 non-null  datetime64[ns]
 8   replyContent          2 non-null      object        
 9   repliedAt             2 non-null      datetime64[ns]
 10  appVersion            3364 non-null   object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 859.5+ KB
Jumlah nilai terdupliasi:  0


In [ ]:
main_df = scrap_df[['reviewId', 'userName', 'content', 'score']]
main_df.head()

,reviewId,userName,content,score
0,c73eebce-095f-4d73-b5aa-f8b698b867f2,Pengguna Google,Game nya sih bagus dah pernah main juga tapi s...,2
1,3035513f-133f-4a7e-826b-af88fcf83454,Pengguna Google,Game ini bagus tapi tolong developer perbanyak...,4
2,0a128133-ead6-4854-b301-d9c32c8ad0c0,Pengguna Google,gamenya seru banget tapi sayangnya setelah men...,5
3,01521727-8847-4c53-bbeb-417e6da8dba2,Pengguna Google,"Tambahin fitur cloudnya dong, biar progressnya...",2
4,f4cf169f-e151-4654-8f86-3a9411936918,Pengguna Google,Ngebug bro .... sesudah isi bulan dan tahun ke...,1


In [ ]:
print(main_df.shape)

(10000, 4)


## Preprocessing Text

### **Case Folding**
Proses mengubah semua huruf dalam teks menjadi huruf kecil atau huruf besar agar konsisten. Misalnya, mengubah "TeKS" menjadi "teks" atau "TEKS".

In [ ]:
main_df.loc[:, 'content'] = main_df.loc[:, 'content'].str.lower()
main_df['content']

0       game nya sih bagus dah pernah main juga tapi s...
1       game ini bagus tapi tolong developer perbanyak...
2       gamenya seru banget tapi sayangnya setelah men...
3       tambahin fitur cloudnya dong, biar progressnya...
4       ngebug bro .... sesudah isi bulan dan tahun ke...
                              ...                        
9995     bagus banget game ini walaupun plant nya sedikit
9996    kerem permainan nya makanya kasih bintang ☆☆☆☆...
9997    karena permainan nya sangat seru dan tidak ada...
9998    aku sudah main game ini udh lama sudah sampai ...
9999    seru sih seru tapi mb nya gede banget tapi tet...
Name: content, Length: 10000, dtype: object

### **Removal Special Characters**
Menghapus karakter khusus atau simbol yang tidak relevan atau tidak diinginkan dari teks.
* Menghapus Angka
* Menghapus Tanda Baca
* Menghapus Garis Baru
* Menghapus Spasi Tambahan Di Awal dan Akhir Teks

In [ ]:
import re

In [ ]:
#Menghapus Angka
for i in range(len(main_df)):
    main_df.loc[i, 'content'] = re.sub(r'\d+', '', main_df.loc[i, 'content'])
main_df['content']

0       game nya sih bagus dah pernah main juga tapi s...
1       game ini bagus tapi tolong developer perbanyak...
2       gamenya seru banget tapi sayangnya setelah men...
3       tambahin fitur cloudnya dong, biar progressnya...
4       ngebug bro .... sesudah isi bulan dan tahun ke...
                              ...                        
9995     bagus banget game ini walaupun plant nya sedikit
9996    kerem permainan nya makanya kasih bintang ☆☆☆☆...
9997    karena permainan nya sangat seru dan tidak ada...
9998    aku sudah main game ini udh lama sudah sampai ...
9999    seru sih seru tapi mb nya gede banget tapi tet...
Name: content, Length: 10000, dtype: object

In [ ]:
# Menghapus Tanda Baca
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
for i in range(len(main_df)):
    text = main_df.loc[i, 'content']
    for char in punc:
        text = text.replace(char, '')
    main_df.loc[i, 'content'] = text
main_df['content']

0       game nya sih bagus dah pernah main juga tapi s...
1       game ini bagus tapi tolong developer perbanyak...
2       gamenya seru banget tapi sayangnya setelah men...
3       tambahin fitur cloudnya dong biar progressnya ...
4       ngebug bro  sesudah isi bulan dan tahun kelahi...
                              ...                        
9995     bagus banget game ini walaupun plant nya sedikit
9996    kerem permainan nya makanya kasih bintang ☆☆☆☆...
9997    karena permainan nya sangat seru dan tidak ada...
9998    aku sudah main game ini udh lama sudah sampai ...
9999    seru sih seru tapi mb nya gede banget tapi tet...
Name: content, Length: 10000, dtype: object

In [ ]:
# Menghapus Mention dan Hastag
for i in range(len(main_df)):
    main_df.loc[i, 'content'] = re.sub(r'@[A-Za-z0-9]+', '', main_df.loc[i, 'content'])
    main_df.loc[i, 'content'] = re.sub(r'#[A-Za-z0-9]+', '', main_df.loc[i, 'content'])
main_df['content']

0      gamenya bagus soalnya ada level nya menantang ...
1      menurut saya game ini sudah cukup bagus saya p...
2      game nya bagusbisa ngajarkan banyak taktik unt...
3      mode survival endless nya tolong di perbaiki k...
4      kalau pas main game ini  aplikasinya suka ngeh...
                             ...                        
195    udah topup pake duit malah coin ngga masukgame...
196    aku sudah kalah kan drzombostapi zombie yeti b...
197    gamenya makin seru dengan fitur terbaru akhirn...
198    tolong ada yang bisa bantu saya sudah  kali in...
199    halo electronic arts pas aku mau masuk di flag...
Name: content, Length: 200, dtype: object

In [ ]:
# Menghapus dan Spasi
main_df['content'] = main_df.loc[:, 'content'].str.strip()
main_df['content']

<ipython-input-15-605d01c1ada3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['content'] = main_df.loc[:, 'content'].str.strip()


0       game nya sih bagus dah pernah main juga tapi s...
1       game ini bagus tapi tolong developer perbanyak...
2       gamenya seru banget tapi sayangnya setelah men...
3       tambahin fitur cloudnya dong biar progressnya ...
4       ngebug bro  sesudah isi bulan dan tahun kelahi...
                              ...                        
9995     bagus banget game ini walaupun plant nya sedikit
9996    kerem permainan nya makanya kasih bintang ☆☆☆☆...
9997    karena permainan nya sangat seru dan tidak ada...
9998    aku sudah main game ini udh lama sudah sampai ...
9999    seru sih seru tapi mb nya gede banget tapi tet...
Name: content, Length: 10000, dtype: object

### **Stopword Removal (Filtering)**
Menghapus kata-kata yang umumnya tidak memberikan nilai tambah dalam analisis teks, seperti "dan", "atau", "yang", dll.

In [ ]:
# !pip install Sastrawi

In [ ]:
import nltk
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import word_tokenize

In [ ]:
factory = StopWordRemoverFactory()
stopwords_sastrawi = factory.get_stop_words()

In [ ]:
def filteringStopwords(text):
  tokens_kata = word_tokenize(text)
  text = [word for word in tokens_kata if word not in stopwords_sastrawi]
  sentence = ' '.join(word for word in text)
  return sentence

In [ ]:
main_df['content'] = main_df['content'].apply(filteringStopwords)
main_df['content']

<ipython-input-31-3c52153c72fa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['content'] = main_df['content'].apply(filteringStopwords)


0      Gamenya bagus soalnya level2 nya menantang min...
1      Menurut game cukup bagus . Saya pas waktu masa...
2      Game nya bagus , ngajarkan banyak taktik melaw...
3      Mode Survival Endless nya perbaiki , keluar sn...
4      Kalau pas main game , aplikasinya suka ngehang...
                             ...                        
195    udah topup pake duit malah coin ngga masuk , g...
196    aku kalah kan dr.zombos.tapi zombie yeti ada.n...
197    Gamenya makin seru Fitur terbaru , akhirnya ko...
198    Tolong bantu ? 5 kali instal tdk mulai permain...
199    Halo , Electronic Arts . Pas aku mau masuk fla...
Name: content, Length: 200, dtype: object

### **Stemming**
Proses menghapus imbuhan dari kata untuk mengembalikannya ke bentuk dasarnya. Misalnya, mengubah "berlari", "berlarian" menjadi "lari".


In [ ]:
# !pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Memecah teks menjadi daftar kata
    words = text.split()

    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]

    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

In [ ]:
main_df['steming'] = main_df['content'].apply(stemmingText)
main_df['content'] = main_df['steming'].apply(toSentence)
main_df['content']

<ipython-input-34-01d85412c3bc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['steming'] = main_df['content'].apply(stemmingText)
<ipython-input-34-01d85412c3bc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['content'] = main_df['steming'].apply(toSentence)


0      g a m e n y a   b a g u s   s o a l   l e v e ...
1      t u r u t   g a m e   c u k u p   b a g u s   ...
2      g a m e   n y a   b a g u s     n g a j a r k ...
3      m o d e   s u r v i v a l   e n d l e s s   n ...
4      k a l a u   p a s   m a i n   g a m e     a p ...
                             ...                        
195    u d a h   t o p u p   p a k e   d u i t   m a ...
196    a k u   k a l a h   k a n   d r   z o m b o s ...
197    g a m e n y a   m a k i n   s e r u   f i t u ...
198    t o l o n g   b a n t u     5   k a l i   i n ...
199    h a l o     e l e c t r o n i c   a r t s     ...
Name: content, Length: 200, dtype: object

### **Tokenizing**
Proses membagi teks menjadi bagian-bagian lebih kecil yang disebut token.


In [ ]:
from nltk.tokenize import word_tokenize
# nltk.download('punkt')

In [ ]:
def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

In [ ]:
main_df['content'] = main_df['content'].apply(tokenizingText)
main_df['content']

### **Lemmatization**
Proses mengubah kata-kata ke bentuk dasarnya (lema) dengan mempertimbangkan konteks dan struktur bahasa. Misalnya, mengubah "menyanyikan" menjadi "nyanyi".